In [35]:
import socketserver

IPS = {}
STONES = 20

class NimTCPHandler(socketserver.BaseRequestHandler):

    def my_move(self):
        remains = IPS[self.ip]
        take = remains % 5
        if take == 0 and take != 4:
            take = 4
        if remains <= 4:
            take = take -1
        return take
    
    def handle(self):
        while True:
            self.ip = self.client_address[0]
            self.data = self.request.recv(1024).strip()
            
            if self.ip not in IPS or IPS[self.ip] <= 0:  # allows to reconnect for the client and restart for new game
                message = "Hello, {}! Starting with {} stones\n".format(self.ip, STONES)
                IPS[self.ip] = STONES
                self.request.send(bytes(message, 'utf8'))
            
            
            move = int(self.data)
            IPS[self.ip] -= move

            message = "IP={} takes {} stones. {} remains\n".format(self.ip, move, IPS[self.ip])
            self.request.send(bytes(message, 'utf8'))

            my = self.my_move()
            IPS[self.ip] -= my
            message = "Server {} takes {} stones, {} remains\n".format(self.ip, my, IPS[self.ip])
            self.request.send(bytes(message, 'utf8'))                


In [36]:
with socketserver.TCPServer(('0.0.0.0', 8989), NimTCPHandler) as server:
    server.serve_forever()

KeyboardInterrupt: 